In [1]:
# librerias

import numpy as np
import pandas as pd
import os
import sys

from sortedcontainers import SortedDict

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
import regex as re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ambar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
size_per_block = 10240*4 # 4 KBytes
num_block = 0
path_block = './blocks/'

In [3]:
class PostingBlock:
    def __init__(self, max_size = 7):
        self.docs = []
        self.next_block = None
        self.max_size = max_size

    def is_full(self):
        return len(self.docs) >= self.max_size

    def add_doc(self, doc_id, tf):
        self.docs.append((doc_id,tf))

In [4]:
def ParseDocs(doc):
    words = []

    col_text = ['lyrics', 'track_name', 'track_artist', 'track_album_name', 'playlist_name', 'playlist_genre', 'playlist_subgenre']
    texto = ''
    for col in col_text:
        texto += ' ' + doc[col]

    texto = texto.lower()

    texto = re.sub(r'[^a-zA-Z0-9_À-ÿ]', ' ', texto)


    if doc['language'] == 'es':
        ln = 'spanish'
    elif doc['language'] == 'tl':
        ln = 'english'
    else:
        ln = 'english'
    
    for word in texto.split():
        
        if word not in stopwords.words(ln):
            words.append(word)

    # aplicar stemming
    stemmer = SnowballStemmer(language=ln)
    words = [stemmer.stem(word) for word in words]

    return words

In [5]:
def WriteBlockToDisk(dictionary, block_id, path='./blocks/'):
    output_file = f"{path}block_{block_id}.txt"
    
    with open(output_file, 'w') as f:
        for term, data in dictionary.items():
            df = data['df']
            postings_list = []
            posting_block = data['posting_block']
            while posting_block is not None:
                postings_list.extend([f"({doc_id}, {tf})" for doc_id, tf in posting_block.docs])
                posting_block = posting_block.next_block
            postings = ', '.join(postings_list)
            f.write(f"{term} (DF: {df}): {postings}\n")
    
    print(f"Block {block_id} written to disk as {output_file}")

In [6]:
def SPIMI_INVERT(token_stream, # recibo una lista de lexemas
                 docId, # el numero del documento
                 num_block, # el numero del bloque en el que escribo
                 size_per_block,
                 path_block = './blocks/'
                 ):
    
    # name_block_archive = path_block + f'block_{num_block}.txt'
    # outputFile = open(name_block_archive, 'w') # NewFIle()

    global global_dictionary, global_block_size

    # dictionary = {} # NewHash() - Diccionario invertido parcial
    # block_size = 0

    
    for token in token_stream:

        if token not in global_dictionary:
            posting_block = PostingBlock()
            global_dictionary[token] = {
                'df': 1,
                'posting_block': posting_block
            }
            posting_block.add_doc(docId, 1)
            global_block_size += sys.getsizeof(token) + sys.getsizeof(global_dictionary[token])
            print(f'word nuevo: {token} se suma block_size: {global_block_size}')
        
        else:
            # posting_list = GetPostingsList(dictionary, term(token))
            posting_block = global_dictionary[token]['posting_block']
            
            doc_found = False
            while posting_block is not None:
                for idx, (doc, tf) in enumerate(posting_block.docs):
                    if doc == docId:
                        posting_block.docs[idx] = (doc, tf + 1)
                        doc_found = True
                        break
                
                if doc_found:
                    break
                
                if posting_block.next_block is None:
                    break
                else:
                    posting_block = posting_block.next_block

            # si el documento no existe    
            if not doc_found:
                global_dictionary[token]['df'] += 1
                
                if posting_block.is_full():
                    new_posting_block = PostingBlock()
                    posting_block.next_block = new_posting_block
                    posting_block = new_posting_block

                posting_block.add_doc(docId, 1)  # Añadir el nuevo doc_id con TF=1
                global_block_size += sys.getsizeof(docId)  # Sumar el tamaño de doc_id

        # Si el bloque alcanza el límite de tamaño, escribir a disco y resetear el diccionario
        if global_block_size >= size_per_block:
            print(f"Writing block {num_block} to disk (block size: {global_block_size} bytes)")
            WriteBlockToDisk(global_dictionary, num_block, path=path_block)
            global_dictionary = {}  # Reiniciar el diccionario global
            global_block_size = 0  # Reiniciar el tamaño del bloque
            num_block += 1  # Incrementar el número de bloque
    
    # sorted_terms = SortTerms(dictionary)
    # sorted_terms = SorTerms(dictionary)
    if global_dictionary:
        WriteBlockToDisk(global_dictionary, num_block, path=path_block)
    # WriteBlocksToDisk(sorted_terms, dictionary,outputFile)
    return num_block


In [63]:
def BSBIndexConstuction(data):

    block_n = 0 # numero de bloque actual 

    f = {} # archivos de bloques
    for doc_id, row  in data.iterrows():

        token_stream = ParseDocs(row)
        num_block = SPIMI_INVERT(token_stream, doc_id, num_block=block_n,  size_per_block=size_per_block, path_block=path_block,)
        block_n = num_block
        f[num_block] = f'{path_block}block_{num_block}.txt'
    
    # f = MergeBlocks(f) 
    return f

In [9]:
# testeo
path = './dataset/'
data_path = path + 'spotify_songs_test.csv'

global_dictionary = {}
global_block_size = 0

data = pd.read_csv(data_path)

In [10]:
BSBIndexConstuction(data)

word nuevo: minsan se suma block_size: 231
word nuevo: pa se suma block_size: 458
word nuevo: nang se suma block_size: 687
word nuevo: ako se suma block_size: 915
word nuevo: napalingon se suma block_size: 1150
word nuevo: hindi se suma block_size: 1380
word nuevo: ko se suma block_size: 1607
word nuevo: alam se suma block_size: 1836
word nuevo: na se suma block_size: 2063
word nuevo: ika se suma block_size: 2291
word nuevo: tutugon se suma block_size: 2523
word nuevo: sa se suma block_size: 2750
word nuevo: mga se suma block_size: 2978
word nuevo: tanong se suma block_size: 3209
word nuevo: ake se suma block_size: 3437
word nuevo: nabitawan se suma block_size: 3671
word nuevo: kung se suma block_size: 3900
word nuevo: ito se suma block_size: 4128
word nuevo: totoo se suma block_size: 4358
word nuevo: pangarap se suma block_size: 4591
word nuevo: ka se suma block_size: 4818
word nuevo: bawat se suma block_size: 5048
word nuevo: sandali se suma block_size: 5280
word nuevo: langit se sum

{0: './blocks/block_0.txt',
 1: './blocks/block_1.txt',
 2: './blocks/block_2.txt',
 4: './blocks/block_4.txt',
 6: './blocks/block_6.txt',
 7: './blocks/block_7.txt',
 8: './blocks/block_8.txt',
 9: './blocks/block_9.txt',
 10: './blocks/block_10.txt',
 11: './blocks/block_11.txt',
 12: './blocks/block_12.txt'}